<a href="https://colab.research.google.com/github/iagombermudez/03MIAR---Algoritmos-de-Optimizacion/blob/main/SEMINARIO/Iago_Berm%C3%BAdez_Trabajo_Pr%C3%A1ctico_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Iago Bermúdez Milleiro  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---/tree/master/TrabajoPractico<br>
Google Colab: https://colab.research.google.com/drive/xxxxxxxxxxxxxxxx <br>
Problema: Configuración de Tribunales

Descripción del problema:

Se precisa configurar tribunales de evaluación para un grupo de 15 alumnos que desean presentar su Trabajo Fin de Máster (TFM).<br/>
- Cada tribunal está compuesto por tres profesores, cada uno desempeñando uno de los siguientes roles:
  - Presidente
  - Secretario
  - Vocal

- Los profesores han indicado su disponibilidad horaria para participar en los tribunales de 15h a 21h durante la semana del 15 al 19 de abril.
- Hay 15 alumnos, por lo que se deben configurar 15 tribunales buscando la configuración más equilibrada posible en cuanto a la cantidad de tribunales asignados a cada profesor, es decir, evitando que un profesor tenga muchos tribunales y otros pocos.<br/>
- Obviamente ningún profesor puede asistir a dos tribunales a la misma fecha/hora y no puede ser convocado a un tribunal al que no tiene disponibilidad.








                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

El espacio de soluciones se puede representar como el conjunto de todas las combinaciones de los miembros de los tribunales. Tratamos las combinaciones en vez de las permutaciones ya que el problem solo pide que asignemos de forma equilibrada la cantidad de tribunales a un profesor, sin importar el número de veces que es presidente, secreatrio o vocal. Por lo tanto un tribunal compuesto por $\{RRD, QYV, MSB\}$ es la misma solución que uno compuesto por $\{QYV, MSB, RRD\}$.

La función objetivo es minimizar la diferencia de las asignaciones de los profesores a los tribunales. Por lo tanto, si tenemos que el conjunto de todas las asingaciones a cada profesor es $X$, entonces la función objetivo sería:<br/>
$Min (Max(X) - Min(X))$<br/>

Teniendo en cuenta que<br/>
$X_{i}$ es el conjunto de asignaciones a un profesor donde $X_{i,j} = 1$ indica la asistencia del profesor y $X_{i,j} = 0$ lo contrario
<br/>$T_{t}$ es el conjunto de roles de un tribunal
<br/>$A_{i}$ es el conjunto de disponibilidad del profesor, donde $A_{i,j} = 1$ indica la disponibilidad del profesor y $A_{i,j} = 0$ lo contrario
<br/>$Q_i$ es el conjunto de roles de un profesor, donde las opciones pueden ser $P$,$S$ o $V$
<br/>Las restricciones del problema serían:<br/>
&emsp;$X_{i,j}\neq1\: si \: A_{i,j}=0$<br/>
&emsp;$1\not\to X_{i,j}\: si\: X_{i,j}=1$<br/>
&emsp;$1\not\to X_{i,j}\: si\:|T_j|=3$<br/>
&emsp;$X_{i,j}=1\lor X_{i,j}=0$<br/>
&emsp;$A_{i,j}=1\lor A_{i,j}=0$<br/>
&emsp;$|X_{i}|=|A_{i}|$<br/>
&emsp;$|Q_{i}|>=1$<br/>
&emsp;$|Q_{i}|<=3$<br/>
&emsp;$1\not\to X_{i,j}\: si\: Q_{i}=\{P\} \land P \in T_j $<br/>
&emsp;$1\not\to X_{i,j}\: si\: Q_{i}=\{S\} \land S \in T_j $<br/>
&emsp;$1\not\to X_{i,j}\: si\: Q_{i}=\{V\} \land V \in T_j $<br/>
&emsp;$1\not\to X_{i,j}\: si\: Q_{i}=\{P,S\} \land P,S \in T_j $<br/>
&emsp;$1\not\to X_{i,j}\: si\: Q_{i}=\{P,V\} \land P,V \in T_j $<br/>
&emsp;$1\not\to X_{i,j}\: si\: Q_{i}=\{V,S\} \land V,S \in T_j $<br/>



#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

Determinar el orden de complejidad para este tipo de problemas puede resultar complejo debido a las diferentes restricciones que incluye. <br/>Si no tenemos en cuenta los roles, el número total de combinaciones para solamente un tribunal es ${n\choose 3}$, siendo $n$ el número de profesores. Por lo tanto en nuestro caso, como $n=10$, tendríamos 120 combinaciones diferentes solo para un tribunal(si además contamos los roles, debemos tener en cuenta que para cada tribunal tendríamos $V_{10, 3}=720$ posibles opciones).
<br/>Por lo tanto, para $k$ tribunales tendríamos ${n\choose 3}^k$ combinaciones distintas. En nuestro caso como $k=15$ tendríamos $120^{15}$ combinaciones distintas para esos tribunales. Eso sin contar que tenemos varias opciones para las horas de los tribunales. En ese caso tendríamos ${35\choose 15}*120^{15}$ opciones diferentes.

Complejidad del problema: Exponencial (al menos $O({n\choose 3}^k)$)<br/>
Espacio de soluciones:${35\choose 15}*{10\choose 3}$ (o si incluimos las permutaciones entre roles, ${35\choose 15}*V_{10, 3}$)

#Diseño
- ¿Que técnica utilizo? ¿Por qué?

Debido a la inmensidad del espacio de soluciones, utilizar un algoritmo por fuerza bruta queda completamente descartado. También debemos descartar los algoritmos deterministas, ya que a medida que aumenta la dimensión también serían indeseables. <br/>
Por ejemplo, podemos pensar en el problema de asignación de tareas de la AG2 en el que utilizamos ramificación y poda. Aquel algoritmo dejaba de ser deseable en una dimensión mucho menor de la que tiene el problema que tenemos entre manos.
Por lo tanto la mejor opción para abarcar estre problema será la de utilizar un algoritmo heurístico. Podemos utilizar por ejemplo una implementación del recocido simulado.

# Implementación

## Funciones básicas

In [112]:
import random
import math
import secrets
import numpy as np

In [113]:
#Respuesta
availability = [
  [0,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,0,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,0,0],
  [1,1,1,1,0,0,0,0,1,1,1,1,0,0,1,0,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
  [0,0,1,1,0,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,1,1,0,1,0,1],
  [1,0,1,0,1,1,0,1,0,0,1,1,1,1,0,0,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,0],
  [1,1,0,1,0,1,1,1,1,1,0,1,1,1,1,0,1,1,0,1,1,0,1,1,1,0,1,1,1,0,1,1,1,1,0],
  [1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,0,0,1,1,1,1,1,0,1,0,1],
  [0,1,1,1,1,1,1,1,1,0,1,1,0,1,0,0,1,1,0,0,1,1,0,0,0,0,1,1,1,1,1,1,1,0,1],
  [1,1,1,1,1,0,0,1,1,0,1,1,1,0,1,1,1,0,0,1,1,0,1,1,1,1,1,1,0,1,1,1,0,1,0],
  [1,0,1,1,0,1,0,0,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1,1],
  [1,1,0,1,1,0,1,1,0,0,0,0,0,1,1,1,0,0,1,1,1,1,0,0,1,1,1,1,1,1,1,0,0,0,1]
]

roles= [
    [1,1,1],
    [1,1,1],
    [1,0,1],
    [0,1,1],
    [1,1,1],
    [1,1,1],
    [0,1,1],
    [0,1,1],
    [1,1,1],
    [1,1,1]
]


In [114]:
def get_role_availability(availability, roles, role_idx):
  return [row if roles[i][role_idx] == 1 else [0] * len(row) for i, row in enumerate(availability)]

def get_presidents_availability(availability, roles):
  return get_role_availability(availability, roles, 0)

def get_secretaries_availability(availability, roles):
  return get_role_availability(availability, roles, 1)

def get_vocals_availability(availability, roles):
  return get_role_availability(availability, roles, 2)

In [115]:
pres_avail =  get_presidents_availability(availability, roles)
secr_avail =  get_secretaries_availability(availability, roles)
vocal_avail =  get_vocals_availability(availability, roles)

In [116]:
def create_empty_solution():
    matrix = [[0 for _ in range(35)] for _ in range(10)]
    return matrix

In [117]:
def get_random_solution(pres_avail, secr_avail, vocal_avail):
  possible_slots = [x for x in range(35)]
  slots = []
  solution = create_empty_solution()
  for i in range(15):
    slot_index = secrets.choice(possible_slots)
    slots.append(slot_index)
    possible_slots.remove(slot_index)

    possible_proffesors = [x for x in range(10)]
    president_index = get_random_professor(possible_proffesors, pres_avail, slot_index)
    possible_proffesors.remove(president_index)
    secretary_index = get_random_professor(possible_proffesors, secr_avail, slot_index)
    possible_proffesors.remove(secretary_index)
    vocal_index = get_random_professor(possible_proffesors, vocal_avail, slot_index)
    possible_proffesors.remove(vocal_index)

    solution[president_index][slot_index] = 1
    solution[secretary_index][slot_index] = 1
    solution[vocal_index][slot_index] = 1
  return solution


In [118]:
def max_dif_between_turns(turns):
  turns_per_professor = []
  for i in range(len(turns)):
    turns_per_professor.append(num_professor_turns(turns, i))
  max_value = max(turns_per_professor)
  min_value = min(turns_per_professor)
  return max_value-min_value

In [119]:
def get_random_professor(possible_professors, role_avail, time):
  indexes = np.where(np.array(role_avail)[:, time] == 1)[0].tolist()
  possible_professors = np.array(possible_professors)
  intersection = np.intersect1d(possible_professors, indexes)
  return secrets.choice(intersection)

In [120]:
def is_available(profesor, time, availability_matrix):
  return availability_matrix[profesor][time] == 1

In [121]:
def num_professor_turns(turns, professor):
  return sum(turns[professor])

In [122]:
def get_professor_name(index):
    mapping = {
        0: 'RRD',
        1: 'QYV',
        2: 'LHL',
        3: 'HLC',
        4: 'MSB',
        5: 'PMQ',
        6: 'QWF',
        7: 'EBB',
        8: 'IOE',
        9: 'IOA',
    }
    return mapping.get(index, "Invalid input")

## Simulated annealing

In [123]:
def probability(T,d):
  return random.random() <  math.exp( -1*d / T)

def lower_temperatura(T):
  return T*0.99

In [124]:
def recocido_simulado(temperature, availability, roles ):
  pres_avail =  get_presidents_availability(availability, roles)
  secr_avail =  get_secretaries_availability(availability, roles)
  vocal_avail =  get_vocals_availability(availability, roles)

  solucion_referencia = get_random_solution(pres_avail, secr_avail, vocal_avail)
  distancia_referencia = max_dif_between_turns(solucion_referencia)

  mejor_solucion = []
  mejor_distancia = 10e100


  N=0
  while temperature > .0001:
    N+=1
    vecina = get_random_solution(pres_avail, secr_avail, vocal_avail)

    distancia_vecina = max_dif_between_turns(vecina)

    if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina

    if distancia_vecina < distancia_referencia or probability(temperature, abs(distancia_referencia - distancia_vecina) ) :
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina

    temperature = lower_temperatura(temperature)
  return mejor_solucion

sol  = recocido_simulado(10000000, availability, roles)

for x in sol:
  print(x)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1]
[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 